# import pandas as pd, numpy as np
from rapidfuzz import process, utils
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('final.csv',index_col = 'Unnamed: 0')
df = data

# Example of extraction of information related to lat,long data
### The address of سمير مجدي مجدي

In [1]:
from geopy.geocoders import Nominatim

coordinates = "31.191351 , 29.923754"

geolocator = Nominatim(user_agent="MyApp")

location = geolocator.reverse(coordinates)

address = location.raw['address']

city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')
elements = ['house_number' , 'road','hamlet','city','state']
readable_address = f"{address['house_number']} {address['road']} {address['hamlet']} {address['city']}"
addd = ''
for e in elements:
    if e in address.keys():
        addd += ' ' + str(address[e])
address

{'house_number': '2',
 'road': 'طريق قناه السويس',
 'hamlet': 'الشاطبى',
 'city': 'الإسكندرية',
 'state': 'الإسكندرية',
 'ISO3166-2-lvl4': 'EG-ALX',
 'postcode': '21557',
 'country': 'مصر',
 'country_code': 'eg'}

### Extracting addresses of all customers

In [ ]:
def get_location(lat,lon):
    try:
        elements = ['house_number' , 'road','hamlet','city']
        full_address = ''
        coordinates = f"{lat} , {lon}"
        geolocator = Nominatim(user_agent="MyApp")
        location = geolocator.reverse(coordinates)
        address = location.raw['address']
        for element in elements:
            if element in address.keys():
                full_address += ' ' + str(address[element])
        return full_address
    
    except:
        
        return ''

In [ ]:
from datetime import datetime, timedelta
#The algorithm returns 5 rows in 2.5 seconds so,,
print(f"The estimated runtime is : {np.round( ( ( ( ( df.shape[0] / 5 ) * 2.5 ) / 60 ) / 60 ) , 2 )} Hours\n")
user = input("Should I proceed? press y to proceed and n to exit: ")
if user == 'y':
    print(f"\nRelax now, The function is working.\n\nTime now is: {datetime.now()}")
    print(f"Comeback at: {datetime.now() + timedelta(hours = np.round( ( ( ( ( df.shape[0] / 5 ) * 2.5 ) / 60 ) / 60 ) , 2 ) )}")
    start = datetime.now()

    df['full_address'] = df.apply(lambda x: get_location(x.latitude , x.longitude) , axis = 1)

    end = datetime.now()
    print(f"\nRuntime of the algorithm is: {end - start}")
else:
    print("\nYou decided not to run the algorithm")


## Similarity Checking Model

In [ ]:
def get_whole_info(name,address):
    return f"{name} {address}"
df.columns

In [ ]:
df['Whole Info'] = df.apply(lambda x: get_whole_info(x.new_full_name,x.full_address), axis = 1)

In [ ]:
from datetime import datetime
start = datetime.now()


name_list = df['Whole Info']
processed_names = [utils.default_process(name) for name in name_list]

for (i, processed_query) in enumerate(processed_names):
    # None is skipped by extractOne, so we set the current element to None and
    # revert this change after the comparision
    processed_names[i] = None
    match = process.extractOne(processed_query, processed_names, processor=None, score_cutoff=70)
    processed_names[i] = processed_query
    if match:
        df.loc[i, 'fuzzy_match'] = name_list[match[2]]
        df.loc[i, 'fuzzy_match_score'] = match[1]
        
        
end = datetime.now()
print(f"Runtime of the algorithm is: {end - start}")

In [ ]:
df.loc[ ( df['fuzzy_match_score'] > 90 ) & ( df['fuzzy_match_score'] != 100 )] 

In [ ]:
df.loc[df['Whole Info'] == 'محمود سليم سليم  شارع مدخل كفر الدوار']
#As you can see they have two different names with two different ids

## For example the following two entries are not the same strings and the result is 'False'
'سليم سليم شارع مدخل كفر الدوار' == 'محمود سليم سليم شارع مدخل كفر الدوار'
### Yet they are so close they scored 95%

In [ ]:
fuzzed_ids = pd.merge(df,df,left_on = 'Whole Info', right_on = 'fuzzy_match',\
                     suffixes = ('_original','_fuzzed'))['retailer_id_fuzzed']
df['fuzzy_match_id'] = fuzzed_ids
df

In [ ]:
df.loc[ ( df['fuzzy_match_score'] > 90 ) & ( df['fuzzy_match_score'] != 100 )] 